In [1]:
!pip install --quiet cartopy netcdf4 earthaccess scipy zipfile36

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 19.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64"

In [2]:
import datetime
from datetime import date

from pathlib import Path

from zipfile import ZipFile

import earthaccess

import xarray as xr

from netCDF4 import Dataset

import requests

import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

import cartopy.feature as cfeature
from cartopy import crs as ccrs

from PIL import Image

import os
from scipy.io import savemat
from google.colab import files as colab
import zipfile
import time

In [3]:
if not os.path.exists('/content/data_files'):
  Path('/content/data_files').mkdir()

if not os.path.exists('/content/temp_data'):
  Path('/content/temp_data').mkdir()

data_path = Path('/content/data_files')
temp_path = Path('/content/temp_data')

In [14]:
auth = earthaccess.login(strategy='interactive')

Enter your Earthdata Login username: tely136
Enter your Earthdata password: ··········


In [18]:
short_name = 'S5P_L2__NO2____HiR'
observation_start = '2023-12-01 00:00:00'  # 'YYYY-MM-DD HH:MM:SS'
observation_end = '2023-12-31 23:59:59' # 'YYYY-MM-DD HH:MM:SS'

'''
lat_min = 38
lat_max = 40

lon_min = -80
lon_max = -75
'''

lat_min = 39
lat_max = 44

lon_min = -77
lon_max = -70
bbox = (lon_min, lat_min, lon_max, lat_max)

# Search NASA Earthdata for TEMPO NO2 granules files
results = earthaccess.search_data(short_name=short_name,
                                  temporal=(observation_start, observation_end),
                                  bounding_box=bbox)

Granules found: 53


In [19]:
overwrite = True
if overwrite == True:
  for file in os.listdir(data_path):
    os.remove(Path(data_path, file))

In [20]:
def download_with_retries(url, retries, delay):
  for attempt in range(retries):
    try:
      response = earthaccess.download(url, local_path=temp_path)
      print(response)
      if response:
        return response

      else:
        print(f"Failed to download {url}")

    except Exception as e:
        print(f"Error downloading {url}: {e}")

    time.sleep(delay)

  print(f"Failed to download {url} after {retries} attempts")
  return None




In [21]:
i = 1
for granule in results:
  for file in os.listdir(temp_path):
      if file.endswith('.nc'):
        os.remove(Path(temp_path,file))

  print(f'Downloading file {i} out of {len(results)}')

  #try:
  #earthaccess.download(granule, local_path=temp_path)
  file_content = download_with_retries(granule, 3, 5)

  lat_slice = slice(lat_min, lat_max)
  lon_slice = slice(lon_min, lon_max)

  file_name = os.listdir(temp_path)
  file_path = os.path.join(temp_path, file_name[0])

  dataset = Dataset(file_path, 'r')
  product = dataset.groups['PRODUCT']

  lat = product.variables['latitude'][:]
  lon = product.variables['longitude'][:]


  region_mask = np.where((lat>=lat_min) & (lat<=lat_max) & (lon>=lon_min) & (lon<=lon_max))

  if any(element.size==0 for element in region_mask):
    print('No data in bounds, skipping iteration')
    continue

  row_min = min(region_mask[1])
  row_max = max(region_mask[1])
  col_min = min(region_mask[2])
  col_max = max(region_mask[2])


  lat = product.variables['latitude']
  lon = product.variables['longitude']
  no2 = product.variables['nitrogendioxide_tropospheric_column']
  qa = product.variables['qa_value']


  lat_subset = lat[0, row_min:row_max+1, col_min:col_max+1].astype(np.float64)
  lon_subset = lon[0, row_min:row_max+1, col_min:col_max+1].astype(np.float64)
  no2_subset = no2[0, row_min:row_max+1, col_min:col_max+1].astype(np.float64)
  qa_subset = qa[0, row_min:row_max+1, col_min:col_max+1].astype(np.float64)

  no2_subset = np.where((no2_subset<0) | (no2_subset>pow(10,36)), np.nan, no2_subset)

  save_data = {
      'no2': no2_subset,
      'lat': lat_subset,
      'lon': lon_subset,
      'qa': qa_subset
  }
  save_name = str(file_name[0]).replace('.nc', '') + '.mat'

  savemat(os.path.join(data_path, save_name), save_data)

  os.remove(file_path)

  i = i+1

 Getting 1 granules, approx download size: 0.57 GB


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

['/content/temp_data/S5P_OFFL_L2__NO2____20231201T155413_20231201T173543_31784_03_020600_20231203T081200.nc']
 Getting 1 granules, approx download size: 0.57 GB


...

['/content/temp_data/S5P_OFFL_L2__NO2____20231230T165029_20231230T183159_32196_03_020600_20240101T091827.nc']
 Getting 1 granules, approx download size: 0.57 GB


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

['/content/temp_data/S5P_OFFL_L2__NO2____20231231T163129_20231231T181259_32210_03_020600_20240102T085146.nc']


In [22]:
zip_filename = 'my_files.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(data_path):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), 'my_files'))

# Download the zip file
colab.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>